In [6]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ['HF_TOKEN'] = user_secrets.get_secret("HF_TOKEN")
os.environ['OPENAI_API_KEY'] = user_secrets.get_secret("OPENAI_API_KEY")

## 모델 finetuning


In [1]:
!ls -al ./yi-ko-6b-text2sql

total 146112
drwxr-xr-x 3 root root      4096 Feb 11 00:37 .
drwxr-xr-x 5 root root      4096 Feb 11 00:37 ..
-rw-r--r-- 1 root root       717 Feb 11 00:37 adapter_config.json
-rw-r--r-- 1 root root 145287696 Feb 11 00:37 adapter_model.safetensors
drwxr-xr-x 4 root root      4096 Feb 11 00:37 autotrain-data
-rw-r--r-- 1 root root      1074 Feb 11 00:37 README.md
-rw-r--r-- 1 root root       573 Feb 11 00:37 special_tokens_map.json
-rw-r--r-- 1 root root      9875 Feb 11 00:37 tokenizer_config.json
-rw-r--r-- 1 root root   4277679 Feb 11 00:37 tokenizer.json
-rw-r--r-- 1 root root      4984 Feb 11 00:37 training_args.bin
-rw-r--r-- 1 root root      1316 Feb 11 00:37 training_params.json


In [2]:
!pip install autotrain-advanced==0.7.77 -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 90.2 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.4/150.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 

### 기존 모델을 새로운 데이터 셋을 튜닝

In [13]:
base_model = 'beomi/Yi-Ko-6B' # base 모델
finetuned_model = 'yi-ko-6b-text2sql' # 교육 후 모델

# --use-peft 양자화 옵션

# --model {base_model} \
# --project-name {finetuned_model} \
# --data-path datasets 경로
# --text-column 학습할 컬럼

!autotrain llm \
--train \
--model {base_model} \
--project-name {finetuned_model} \
--data-path datasets/ \
--text-column prompt \
--lr 2e-4 \
--batch-size 2 \
--epochs 1 \
--block-size 1024 \
--warmup-ratio 0.1 \
--lora-r 16 \
--lora-alpha 32 \
--lora-dropout 0.05 \
--weight-decay 0.01 \
--gradient-accumulation 8 \
--mixed-precision fp16 \
--use-peft \
--quantization int4 \
--trainer sft

INFO     | 2025-02-10 03:50:42 | autotrain.cli.run_llm:run:343 - Running LLM
WARNING  | 2025-02-10 03:50:42 | autotrain.trainers.common:__init__:180 - Parameters supplied but not used: train, backend, inference, deploy, func, version, config
Saving the dataset (1/1 shards): 100%|█| 33876/33876 [00:00<00:00, 400147.13 exa
Saving the dataset (1/1 shards): 100%|█| 33876/33876 [00:00<00:00, 417345.89 exa
INFO     | 2025-02-10 03:50:45 | autotrain.backend:create:300 - Starting local training...
INFO     | 2025-02-10 03:50:45 | autotrain.commands:launch_command:327 - ['accelerate', 'launch', '--multi_gpu', '--num_machines', '1', '--num_processes', '2', '--mixed_precision', 'fp16', '-m', 'autotrain.trainers.clm', '--training_config', 'yi-ko-6b-text2sql/training_params.json']
INFO     | 2025-02-10 03:50:45 | autotrain.commands:launch_command:328 - {'model': 'beomi/Yi-Ko-6B', 'project_name': 'yi-ko-6b-text2sql', 'data_path': 'yi-ko-6b-text2sql/autotrain-data', 'train_split': 'train', 'valid_spl

In [4]:
!pip install transformers==4.40.1 bitsandbytes==0.43.1 accelerate==0.29.3 datasets==2.19.0 tiktoken==0.6.0 -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 65.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 79.1 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.0 which is incompatible.
gcsfs 2024.10.0 r

### 데이터셋
- 실습 데이터 (GPT로 생성) : https://huggingface.co/datasets/shangrilar/ko_text2sql/viewer/origin
- 게임 DB에 맞게 형성

### 데이터 로딩
- 영어 text2sql : https://huggingface.co/datasets/Salesforce/wikisql
- 한글 text2sql : https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=data&dataSetSn=71351


In [3]:
!pwd

/kaggle/working


In [7]:
!ls -la ../input/generativellm-hf-dataset/datasets

total 50680
drwxr-xr-x 2 nobody nogroup        0 Feb 10 03:35 .
drwxr-xr-x 3 nobody nogroup        0 Feb 10 03:35 ..
-rw-r--r-- 1 nobody nogroup   130329 Feb 10 03:35 test.csv
-rw-r--r-- 1 nobody nogroup 51762590 Feb 10 03:35 train.csv


In [8]:
!mkdir ./datasets

In [9]:
!ls -la 

total 16
drwxr-xr-x 4 root root 4096 Feb 10 03:38 .
drwxr-xr-x 5 root root 4096 Feb 10 03:30 ..
drwxr-xr-x 2 root root 4096 Feb 10 03:38 datasets
drwxr-xr-x 2 root root 4096 Feb 10 03:31 .virtual_documents


In [10]:
!cp ../input/generativellm-hf-dataset/datasets/train.csv ./datasets

In [12]:
!ls -la ./datasets

total 50560
drwxr-xr-x 2 root root     4096 Feb 10 03:39 .
drwxr-xr-x 4 root root     4096 Feb 10 03:38 ..
-rw-r--r-- 1 root root 51762590 Feb 10 03:39 train.csv


In [13]:
# huggingface dataset to pandas dataframe
text2sql_train_df = text2sql_dataset['train'].to_pandas()
text2sql_test_df = text2sql_dataset['test'].to_pandas()
text2sql_train_df.head()
text2sql_test_df.head()

,db_id,context,question,answer
0,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,각 보상 아이템별로 보상 경험치의 합을 구해줘,"SELECT reward_items, SUM(reward_experience) AS..."
1,1,CREATE TABLE players (\n player_id INT PRIMAR...,사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.,SELECT COUNT(*) FROM players WHERE username LI...
2,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,퀘스트 진행 상황이 100%인 퀘스트의 이름과 보상 경험치는 얼마인가요?,"SELECT q.name, q.reward_experience FROM quests..."
3,1,CREATE TABLE characters (\n character_id INT ...,경험이 5000000 이상이거나 직업이 전사인 캐릭터들의 이름은 무엇인가,SELECT name FROM characters WHERE experience >...
4,1,CREATE TABLE characters (\n character_id INT ...,레벨이 20 이상인 플레이어의 캐릭터 이름과 해당 캐릭터의 스킬 이름을 알아보세요.,"SELECT C.name, ST.skill_name FROM characters A..."


In [15]:
text2sql_train_df = text2sql_train_df.dropna()
text2sql_train_df = text2sql_train_df.query('db_id != 1') # 정상 query 만 사용
text2sql_train_df.head()

,db_id,context,question,answer
2388,2,"CREATE TABLE messages (\n ""message_id"" SERI...",messages 테이블에서 모든 데이터를 조회해줘,SELECT * FROM messages;
2389,2,"CREATE TABLE messages (\n ""message_id"" SERI...",보낸 사람의 ID가 15보다 큰 메시지들의 수를 알려줘,SELECT COUNT(*) FROM messages WHERE sender_id ...
2390,2,"CREATE TABLE messages (\n ""message_id"" SERI...",읽지 않은 메시지들은 몇 개인가요?,SELECT COUNT(*) FROM messages WHERE read = FALSE;
2391,2,"CREATE TABLE messages (\n ""message_id"" SERI...",대화 ID가 42인 대화에서 보낸 메시지들은 몇 개인가요?,SELECT COUNT(*) FROM messages WHERE conversati...
2392,2,"CREATE TABLE messages (\n ""message_id"" SERI...",보낸 시간이 2023-01-01 이후인 메시지의 수를 구해줘,SELECT COUNT(*) FROM messages WHERE timestamp ...


In [18]:
def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.
### DDL:
{ddl}
### Question:
{question}
### SQL:
{query}"""
    return prompt

In [19]:
make_prompt(text2sql_train_df['context'],text2sql_train_df['question'],text2sql_train_df['answer'])

'당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.\n### DDL:\n2388     CREATE TABLE messages (\\n    "message_id" SERI...\n2389     CREATE TABLE messages (\\n    "message_id" SERI...\n2390     CREATE TABLE messages (\\n    "message_id" SERI...\n2391     CREATE TABLE messages (\\n    "message_id" SERI...\n2392     CREATE TABLE messages (\\n    "message_id" SERI...\n                               ...                        \n38241    CREATE TABLE users (\\n    "user_id" SERIAL PRI...\n38242    CREATE TABLE users (\\n    "user_id" SERIAL PRI...\n38243    CREATE TABLE users (\\n    "user_id" SERIAL PRI...\n38244    CREATE TABLE users (\\n    "user_id" SERIAL PRI...\n38245    CREATE TABLE users (\\n    "user_id" SERIAL PRI...\nName: context, Length: 33876, dtype: object\n### Question:\n2388                           messages 테이블에서 모든 데이터를 조회해줘\n2389                        보낸 사람의 ID가 15보다 큰 메시지들의 수를 알려줘\n2390                                   읽지 않은 메시지들은 몇 개인가요?\n2391  

In [21]:
# 필요한 컬럼 추출해 프롬프트 명령어 만들기
from tqdm import tqdm

for idx, row in tqdm(text2sql_train_df.iterrows(),
                     total = len(text2sql_train_df), 
                     desc='Generating prompt'): # DataFrame row return
    prompt_command = make_prompt(row['context'], 
                         row['question'], 
                         row['answer']
                        )
    text2sql_train_df.loc[idx, 'prompt'] = prompt_command
    pass
text2sql_train_df.head()

Generating prompt: 100%|██████████| 33876/33876 [00:07<00:00, 4575.38it/s]


,db_id,context,question,answer,prompt
2388,2,"CREATE TABLE messages (\n ""message_id"" SERI...",messages 테이블에서 모든 데이터를 조회해줘,SELECT * FROM messages;,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2389,2,"CREATE TABLE messages (\n ""message_id"" SERI...",보낸 사람의 ID가 15보다 큰 메시지들의 수를 알려줘,SELECT COUNT(*) FROM messages WHERE sender_id ...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2390,2,"CREATE TABLE messages (\n ""message_id"" SERI...",읽지 않은 메시지들은 몇 개인가요?,SELECT COUNT(*) FROM messages WHERE read = FALSE;,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2391,2,"CREATE TABLE messages (\n ""message_id"" SERI...",대화 ID가 42인 대화에서 보낸 메시지들은 몇 개인가요?,SELECT COUNT(*) FROM messages WHERE conversati...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2392,2,"CREATE TABLE messages (\n ""message_id"" SERI...",보낸 시간이 2023-01-01 이후인 메시지의 수를 구해줘,SELECT COUNT(*) FROM messages WHERE timestamp ...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...


In [23]:
!mkdir ./datasets

In [24]:
# 재사용 위한 저장
dataset_path = f'datasets'
text2sql_train_df.to_csv(f'{dataset_path}/train.csv')

In [26]:
for idx, row in tqdm(text2sql_test_df.iterrows(),
                     total = len(text2sql_test_df), 
                     desc='Generating prompt'): # DataFrame row return
    prompt_command = make_prompt(row['context'], 
                         row['question'], 
                         ''
                        )
    text2sql_test_df.loc[idx, 'prompt'] = prompt_command
    pass
text2sql_test_df.head()

Generating prompt: 100%|██████████| 112/112 [00:00<00:00, 2057.04it/s]


,db_id,context,question,answer,prompt
0,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,각 보상 아이템별로 보상 경험치의 합을 구해줘,"SELECT reward_items, SUM(reward_experience) AS...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
1,1,CREATE TABLE players (\n player_id INT PRIMAR...,사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.,SELECT COUNT(*) FROM players WHERE username LI...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,퀘스트 진행 상황이 100%인 퀘스트의 이름과 보상 경험치는 얼마인가요?,"SELECT q.name, q.reward_experience FROM quests...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
3,1,CREATE TABLE characters (\n character_id INT ...,경험이 5000000 이상이거나 직업이 전사인 캐릭터들의 이름은 무엇인가,SELECT name FROM characters WHERE experience >...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
4,1,CREATE TABLE characters (\n character_id INT ...,레벨이 20 이상인 플레이어의 캐릭터 이름과 해당 캐릭터의 스킬 이름을 알아보세요.,"SELECT C.name, ST.skill_name FROM characters A...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...


In [28]:
text2sql_test_df['prompt'][0]

'당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.\n### DDL:\nCREATE TABLE quests (\n  quest_id INT PRIMARY KEY AUTO_INCREMENT,\n  name VARCHAR(255) NOT NULL,\n  description TEXT,\n  reward_experience INT NOT NULL,\n  reward_items VARCHAR(255)\n);\n### Question:\n각 보상 아이템별로 보상 경험치의 합을 구해줘\n### SQL:\n'

In [29]:
text2sql_test_df.to_csv(f'{dataset_path}/test.csv')

### 기존 참조 모델과 튜닝 모델 머지

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, PeftModel
base_model = 'beomi/Yi-Ko-6B'
finetuned_model = 'yi-ko-6b-text2sql'
model_name = base_model
device_map = {"": 0}

# LoRA와 기초 모델 파라미터 합치기
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, finetuned_model)
model = model.merge_and_unload()

# 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 20.12 MiB is free. Process 3552 has 14.72 GiB memory in use. Of the allocated memory 14.58 GiB is allocated by PyTorch, and 17.24 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [4]:
# 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'


tokenizer_config.json:   0%|          | 0.00/9.51k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

In [7]:
# finetuning 된 모델 hugging face에 등록
model.push_to_hub(finetuned_model, use_temp_dir=False)

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/HueyVault/yi-ko-6b-text2sql/commit/6bac3cc886309f3bfd6a5236bfd5628693b3d590', commit_message='Upload LlamaForCausalLM', commit_description='', oid='6bac3cc886309f3bfd6a5236bfd5628693b3d590', pr_url=None, repo_url=RepoUrl('https://huggingface.co/HueyVault/yi-ko-6b-text2sql', endpoint='https://huggingface.co', repo_type='model', repo_id='HueyVault/yi-ko-6b-text2sql'), pr_revision=None, pr_num=None)

In [8]:
# 사용한 tokenizer를 higging face 등록
tokenizer.push_to_hub(finetuned_model, use_temp_dir=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/HueyVault/yi-ko-6b-text2sql/commit/d4c8f046347115e42569ec9edc4dbb31fb2444b0', commit_message='Upload tokenizer', commit_description='', oid='d4c8f046347115e42569ec9edc4dbb31fb2444b0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/HueyVault/yi-ko-6b-text2sql', endpoint='https://huggingface.co', repo_type='model', repo_id='HueyVault/yi-ko-6b-text2sql'), pr_revision=None, pr_num=None)